In [1]:
import numpy as np
import scipy as sp
from scipy import optimize,special,integrate,stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import copy

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
Λ = sp.stats.beta.ppf(np.linspace(0,1,1000),1,10)
A = np.diag(Λ)

n = len(Λ)
b = np.ones(n,dtype=np.double)
b /= np.linalg.norm(b)

f = lambda x: (x>=0)
fAb = f(Λ)*b

In [4]:
K = 100
Q,(a_,b_) = lanczos_reorth(A,b,K,reorth=K)

In [5]:
i = 10

shifts = np.linspace(-.1,.5,1000)

spec_OR = np.zeros_like(shifts)
spec_FA = np.zeros_like(shifts)
spec_HA = np.zeros_like(shifts)

for j,c in enumerate(shifts):


    T = np.diag(a_[:i]-c) + np.diag(b_[:i-1],1) + np.diag(b_[:i-1],-1)
    theta,S = sp.linalg.eigh(T)
    
    spec_FA[j] = S[0]@((theta)/np.sqrt((theta)**2)*S[0])

    TT_ = T@T 
    TT_[-1,-1] += b_[i-1]**2
    theta,S = sp.linalg.eigh(TT_)

    spec_OR[j] = (S[0]@((1/np.sqrt(theta))*(S.T@T[0])))

    
    ek1 = np.zeros(i)
    ek1[-1] = 1
    e0 = np.zeros(i)
    e0[0] = 1
    
    z = b_[i-1]**2*np.linalg.solve(T,ek1)
    T_ = np.copy(T)
    T_[:,-1] += z
    theta,S = sp.linalg.eig(T_)
    theta = np.real(theta)
    spec_HA[j] = S[0]@((theta)/np.sqrt((theta)**2)*np.linalg.solve(S,e0))
    

In [6]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))
ax = fig.add_axes([left, bottom, width, height])




ax.plot(shifts,(1-spec_FA)/2,**line_styles_nomarker['l2'])
ax.plot(shifts,(1-spec_FA)/2,**line_styles_nomarker1['l2'])

ax.plot(shifts,(1-spec_HA)/2,**line_styles_nomarker['l3'])
ax.plot(shifts,(1-spec_HA)/2,**line_styles_nomarker1['l3'])

ax.plot(shifts,(1-spec_OR)/2,**line_styles_nomarker['l1'])


ax.plot(np.linspace(.0,.5,100),sp.stats.beta.cdf(np.linspace(.0,.5,100),1,10),\
           **line_styles_nomarker['l4'])
ax.plot(np.linspace(.0,.5,100),sp.stats.beta.cdf(np.linspace(.0,.5,100),1,10),\
           **line_styles_nomarker1['l4'])

plt.grid(True,linestyle=':',linewidth=.5)

#plt.ylim(1e-14,5e-1)

ax.set_xlim(-.05,.5)
ax.set_yticks([0,.2,.4,.6,.8,1])
ax.set_xlabel('value of parameter $c$')



axins = inset_axes(ax, "100%","100%", loc='upper left', bbox_to_anchor=(0.55,.15,.4,.6), bbox_transform=ax.transAxes) # zoom = 6


axins.plot(shifts,(1-spec_FA)/2,**line_styles_nomarker['l2'])
axins.plot(shifts,(1-spec_FA)/2,**line_styles_nomarker1['l2'])

axins.plot(shifts,(1-spec_HA)/2,**line_styles_nomarker['l3'])
axins.plot(shifts,(1-spec_HA)/2,**line_styles_nomarker1['l3'])

axins.plot(shifts,(1-spec_OR)/2,**line_styles_nomarker['l1'])

axins.plot(np.linspace(.0,.05,100),sp.stats.beta.cdf(np.linspace(.0,.05,100),1,10),\
           **line_styles_nomarker['l4'])
axins.plot(np.linspace(.0,.05,100),sp.stats.beta.cdf(np.linspace(.0,.05,100),1,10),\
           **line_styles_nomarker1['l4'])


# sub region of the original image
x1, x2, y1, y2 = -.02,.05,-.02,.5
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([0,x2])
axins.set_yticks([0,y2])

plt.savefig('imgs/ch6_sign_spec.pdf')
#plt.close()